In [1]:
!pip install PyGithub

In [1]:
from github import Github
import os
import base64
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px
from ipywidgets import interact
import datetime as dt
sns.set()

In [2]:
with open("github.txt") as myfile:
    firstNlines=myfile.readlines()[0:2]
myfile.close()

In [3]:
g = Github(firstNlines[0].strip(), firstNlines[1])

In [4]:
repo=g.get_repo('CSSEGISandData/COVID-19')

In [5]:
contents = repo.get_contents("")

In [6]:
contents[3]

ContentFile(path="csse_covid_19_data")

In [7]:
def get_sha_for_tag(repository, tag):
    """
    Returns a commit PyGithub object for the specified repository and tag.
    """
    branches = repository.get_branches()
    matched_branches = [match for match in branches if match.name == tag]
    if matched_branches:
        return matched_branches[0].commit.sha

    tags = repository.get_tags()
    matched_tags = [match for match in tags if match.name == tag]
    if not matched_tags:
        raise ValueError('No Tag or Branch exists with that name')
    return matched_tags[0].commit.sha

In [8]:
def download_directory(repository, sha, server_path):
    """
    Download all contents at server_path with commit tag sha in
    the repository.
    """
    contents = repository.get_contents(server_path, ref=sha)

    for content in contents:
        print("Processing %s" % content.path)
        if content.type == 'dir':
            download_directory(repository, sha, content.path)
        else:
            try:
                path = content.path
                file_content = repository.get_contents(path, ref=sha)
                file_data = base64.b64decode(file_content.content).decode('ascii')
                file_out = open(content.name, "w")
                file_out.write(file_data)
                file_out.close()
            except:
                pass

In [9]:
sha = get_sha_for_tag(repo, 'master')

In [10]:
download_directory(repo, sha, 'csse_covid_19_data/csse_covid_19_daily_reports')

Processing csse_covid_19_data/csse_covid_19_daily_reports/.gitignore
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-22-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-23-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-24-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-25-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-26-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-27-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-28-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-29-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-30-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/01-31-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/02-01-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_reports/02-02-2020.csv
Processing csse_covid_19_data/csse_covid_19_daily_repor

In [11]:
df = pd.read_csv(r'C:\Users\brian\projects\COVID-19\csse_daily_reports\02-01-2020.csv')

In [12]:
from os import listdir
from os.path import isfile, join
mypath = r'C:\Users\brian\projects\COVID-19\csse_daily_reports\\'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [13]:
def is_non_zero_file(fpath):
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0

In [14]:
nonzero_files = []
for file in onlyfiles:
    if is_non_zero_file(mypath + file) and file[-3:]=='csv':
        nonzero_files.append(mypath + file)

In [15]:
nonzero_files

['C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-01-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-02-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-03-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-04-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-05-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-06-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-07-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-08-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-09-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-10-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-11-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_reports\\\\02-12-2020.csv',
 'C:\\Users\\brian\\projects\\COVID-19\\csse_daily_r

In [16]:
df_list = []
for file in nonzero_files:
    df_1 = pd.read_csv(file)
    df_1['filename'] = file[-15:]
    df_list.append(df_1)  

In [17]:
df = pd.concat(df_list, ignore_index=True)

C:\Users\brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4226 entries, 0 to 4225
Data columns (total 9 columns):
Confirmed         4226 non-null int64
Country/Region    4226 non-null object
Deaths            4226 non-null int64
Last Update       4226 non-null object
Latitude          1901 non-null float64
Longitude         1901 non-null float64
Province/State    2621 non-null object
Recovered         4226 non-null int64
filename          4226 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 297.3+ KB


In [19]:
df['Date'] = pd.to_datetime(df['Last Update'])

In [20]:
df['Diamond Princess'] = df['Province/State'].str.contains('Diamond Princess')

In [21]:
df.loc[df['Province/State']=='From Diamond Princess']

,Confirmed,Country/Region,Deaths,Last Update,Latitude,Longitude,Province/State,Recovered,filename,Date,Diamond Princess
1499,4,Australia,0,2020-02-21T23:03:13,NaN,NaN,From Diamond Princess,0,\02-21-2020.csv,2020-02-21 23:03:13,True
1518,1,Israel,0,2020-02-21T15:33:03,NaN,NaN,From Diamond Princess,0,\02-21-2020.csv,2020-02-21 15:33:03,True
1580,7,Australia,0,2020-02-22T17:03:05,NaN,NaN,From Diamond Princess,0,\02-22-2020.csv,2020-02-22 17:03:05,True
1602,1,Israel,0,2020-02-22T20:53:02,NaN,NaN,From Diamond Princess,0,\02-22-2020.csv,2020-02-22 20:53:02,True
1664,7,Australia,0,2020-02-22T17:03:05,NaN,NaN,From Diamond Princess,0,\02-23-2020.csv,2020-02-22 17:03:05,True
1686,1,Israel,0,2020-02-22T20:53:02,NaN,NaN,From Diamond Princess,0,\02-23-2020.csv,2020-02-22 20:53:02,True
1749,7,Australia,0,2020-02-22T17:03:05,NaN,NaN,From Diamond Princess,0,\02-24-2020.csv,2020-02-22 17:03:05,True
1773,1,Israel,0,2020-02-22T20:53:02,NaN,NaN,From Diamond Princess,0,\02-24-2020.csv,2020-02-22 20:53:02,True
1841,7,Australia,0,2020-02-22T17:03:05,NaN,NaN,From Diamond Princess,0,\02-25-2020.csv,2020-02-22 17:03:05,True
1867,1,Israel,0,2020-02-22T20:53:02,NaN,NaN,From Diamond Princess,0,\02-25-2020.csv,2020-02-22 20:53:02,True


In [22]:
df.drop(columns=['filename'], inplace=True)

In [23]:
df = df.drop_duplicates()

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2421 entries, 0 to 4225
Data columns (total 10 columns):
Confirmed           2421 non-null int64
Country/Region      2421 non-null object
Deaths              2421 non-null int64
Last Update         2421 non-null object
Latitude            1118 non-null float64
Longitude           1118 non-null float64
Province/State      1602 non-null object
Recovered           2421 non-null int64
Date                2421 non-null datetime64[ns]
Diamond Princess    1602 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 208.1+ KB


In [25]:
df['Province/State'] = df['Province/State'].str.replace(' \(From Diamond Princess\)', '').replace('None', 'From Diamond Princess')

In [26]:
df.head()

,Confirmed,Country/Region,Deaths,Last Update,Latitude,Longitude,Province/State,Recovered,Date,Diamond Princess
0,7153,Mainland China,249,2/1/2020 11:53,NaN,NaN,Hubei,168,2020-02-01 11:53:00,False
1,599,Mainland China,0,2/1/2020 10:53,NaN,NaN,Zhejiang,21,2020-02-01 10:53:00,False
2,535,Mainland China,0,2/1/2020 14:23,NaN,NaN,Guangdong,14,2020-02-01 14:23:00,False
3,422,Mainland China,2,2/1/2020 1:52,NaN,NaN,Henan,3,2020-02-01 01:52:00,False
4,389,Mainland China,0,2/1/2020 11:03,NaN,NaN,Hunan,8,2020-02-01 11:03:00,False


In [27]:
df['DateTime'] = df['Date']

In [28]:
df['Date'] = df['DateTime'].apply(lambda x: x.date())

In [29]:
df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))

In [47]:
df['Date'] = df['Date'].apply(lambda x: str(x))

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2421 entries, 0 to 4225
Data columns (total 13 columns):
Confirmed           2421 non-null int64
Country/Region      2421 non-null object
Deaths              2421 non-null int64
Last Update         2421 non-null object
Latitude            1118 non-null float64
Longitude           1118 non-null float64
Province/State      1602 non-null object
Recovered           2421 non-null int64
Date                2421 non-null object
Diamond Princess    1602 non-null object
DateTime            2421 non-null datetime64[ns]
Active              2421 non-null int64
Log Confirmed       2421 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(4), object(5)
memory usage: 264.8+ KB


In [30]:
df.head()

,Confirmed,Country/Region,Deaths,Last Update,Latitude,Longitude,Province/State,Recovered,Date,Diamond Princess,DateTime
0,7153,Mainland China,249,2/1/2020 11:53,NaN,NaN,Hubei,168,2020-02-01,False,2020-02-01 11:53:00
1,599,Mainland China,0,2/1/2020 10:53,NaN,NaN,Zhejiang,21,2020-02-01,False,2020-02-01 10:53:00
2,535,Mainland China,0,2/1/2020 14:23,NaN,NaN,Guangdong,14,2020-02-01,False,2020-02-01 14:23:00
3,422,Mainland China,2,2/1/2020 1:52,NaN,NaN,Henan,3,2020-02-01,False,2020-02-01 01:52:00
4,389,Mainland China,0,2/1/2020 11:03,NaN,NaN,Hunan,8,2020-02-01,False,2020-02-01 11:03:00


In [31]:
df['Active'] = df['Confirmed']-df['Deaths']-df['Recovered']

In [32]:
df.loc[df['Country/Region']=='South Africa']

,Confirmed,Country/Region,Deaths,Last Update,Latitude,Longitude,Province/State,Recovered,Date,Diamond Princess,DateTime,Active
3048,1,South Africa,0,2020-03-05T13:53:03,-30.5595,22.9375,NaN,0,2020-03-05,NaN,2020-03-05 13:53:03,1
3236,1,South Africa,0,2020-03-06T15:43:02,-30.5595,22.9375,NaN,0,2020-03-06,NaN,2020-03-06 15:43:02,1
3641,3,South Africa,0,2020-03-08T10:23:04,-30.5595,22.9375,NaN,0,2020-03-08,NaN,2020-03-08 10:23:04,3
4140,7,South Africa,0,2020-03-10T05:13:07,-30.5595,22.9375,NaN,0,2020-03-10,NaN,2020-03-10 05:13:07,7


In [43]:
df['Log Confirmed'] = np.log(df['Confirmed']+1)

In [69]:
grouped_df = df.groupby(['Date', 'Country/Region']).agg({'Confirmed': 'sum', 'Recovered': 'sum',
                                                    'Active': 'sum', 'Deaths': 'sum'}).reset_index()

In [73]:
grouped_df['Log Confirmed'] = np.log(grouped_df['Confirmed']+1)

In [71]:
grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 6 columns):
Date              990 non-null object
Country/Region    990 non-null object
Confirmed         990 non-null int64
Recovered         990 non-null int64
Active            990 non-null int64
Deaths            990 non-null int64
dtypes: int64(4), object(2)
memory usage: 46.5+ KB


1. Check for each country if it has entries for every date
2. If country is missing entry for that date, use most previous entry

In [89]:
country_list = grouped_df['Country/Region'].unique()

In [165]:
temp_dict = {}
for country in country_list:
    date_list = list(grouped_df[grouped_df['Country/Region']==country]['Date'])
    missing_dates = list(set(china_dates)-set(date_list))
    temp_dict[country]=missing_dates
    
#     df_temp = pd.DataFrame({'Country/Region':country, 'Date':missing_dates})
#     grouped_df_2.append(df_temp, ignore_index=True)
    

In [174]:
missing_dates_df = pd.DataFrame.from_dict(temp_dict, orient='index')

In [175]:
missing_dates_df.reset_index()

,index,0,1,2,3,4,5,6,7,8,...,29,30,31,32,33,34,35,36,37,38
0,Cambodia,2020-02-04 00:00:00,2020-02-11 00:00:00,2020-02-07 00:00:00,2020-02-01 00:00:00,2020-03-04 00:00:00,2020-02-03 00:00:00,2020-02-17 00:00:00,2020-03-02 00:00:00,2020-02-15 00:00:00,...,2020-02-05 00:00:00,2020-02-26 00:00:00,2020-02-28 00:00:00,2020-02-18 00:00:00,2020-03-03 00:00:00,2020-03-05 00:00:00,2020-02-02 00:00:00,2020-02-10 00:00:00,None,None
1,Finland,2020-02-04 00:00:00,2020-02-11 00:00:00,2020-02-07 00:00:00,2020-02-01 00:00:00,2020-03-04 00:00:00,2020-02-03 00:00:00,2020-02-17 00:00:00,2020-03-02 00:00:00,2020-02-15 00:00:00,...,2020-02-10 00:00:00,None,None,None,None,None,None,None,None,None
2,India,2020-02-04 00:00:00,2020-02-11 00:00:00,2020-02-07 00:00:00,2020-02-01 00:00:00,2020-02-17 00:00:00,2020-02-15 00:00:00,2020-03-01 00:00:00,2020-02-06 00:00:00,2020-02-22 00:00:00,...,None,None,None,None,None,None,None,None,None,None
3,Italy,2020-02-04 00:00:00,2020-02-11 00:00:00,2020-02-01 00:00:00,2020-02-03 00:00:00,2020-02-17 00:00:00,2020-02-15 00:00:00,2020-02-06 00:00:00,2020-02-14 00:00:00,2020-02-19 00:00:00,...,None,None,None,None,None,None,None,None,None,None
4,Macau,2020-02-11 00:00:00,2020-02-07 00:00:00,2020-02-01 00:00:00,2020-03-04 00:00:00,2020-02-03 00:00:00,2020-02-17 00:00:00,2020-03-02 00:00:00,2020-02-15 00:00:00,2020-03-01 00:00:00,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,Russian Federation,2020-02-04 00:00:00,2020-02-11 00:00:00,2020-02-07 00:00:00,2020-02-01 00:00:00,2020-03-04 00:00:00,2020-02-03 00:00:00,2020-02-17 00:00:00,2020-03-02 00:00:00,2020-02-15 00:00:00,...,2020-02-05 00:00:00,2020-01-31 00:00:00,2020-02-26 00:00:00,2020-02-28 00:00:00,2020-03-08 00:00:00,2020-02-18 00:00:00,2020-03-03 00:00:00,2020-03-05 00:00:00,2020-02-02 00:00:00,2020-02-10 00:00:00
125,Saint Martin,2020-02-04 00:00:00,2020-02-11 00:00:00,2020-02-07 00:00:00,2020-02-01 00:00:00,2020-03-04 00:00:00,2020-02-03 00:00:00,2020-02-17 00:00:00,2020-03-02 00:00:00,2020-02-15 00:00:00,...,2020-02-05 00:00:00,2020-01-31 00:00:00,2020-02-26 00:00:00,2020-02-28 00:00:00,2020-03-08 00:00:00,2020-02-18 00:00:00,2020-03-03 00:00:00,2020-03-05 00:00:00,2020-02-02 00:00:00,2020-02-10 00:00:00
126,Taipei and environs,2020-02-04 00:00:00,2020-02-11 00:00:00,2020-02-07 00:00:00,2020-02-01 00:00:00,2020-03-04 00:00:00,2020-02-03 00:00:00,2020-02-17 00:00:00,2020-03-02 00:00:00,2020-02-15 00:00:00,...,2020-02-05 00:00:00,2020-01-31 00:00:00,2020-02-26 00:00:00,2020-02-28 00:00:00,2020-03-08 00:00:00,2020-02-18 00:00:00,2020-03-03 00:00:00,2020-03-05 00:00:00,2020-02-02 00:00:00,2020-02-10 00:00:00
127,Viet Nam,2020-02-04 00:00:00,2020-02-11 00:00:00,2020-02-07 00:00:00,2020-02-01 00:00:00,2020-03-04 00:00:00,2020-02-03 00:00:00,2020-02-17 00:00:00,2020-03-02 00:00:00,2020-02-15 00:00:00,...,2020-02-05 00:00:00,2020-01-31 00:00:00,2020-02-26 00:00:00,2020-02-28 00:00:00,2020-03-08 00:00:00,2020-02-18 00:00:00,2020-03-03 00:00:00,2020-03-05 00:00:00,2020-02-02 00:00:00,2020-02-10 00:00:00


In [177]:
pd.melt(missing_dates_df, id_vars=missing_dates_df.index, value_vars=missing_dates_df.columns)

KeyError: "The following 'id_vars' are not present in the DataFrame: [' Azerbaijan', 'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bhutan', 'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada', 'Channel Islands', 'Chile', 'Colombia', 'Costa Rica', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Estonia', 'Faroe Islands', 'Finland', 'France', 'French Guiana', 'Georgia', 'Germany', 'Gibraltar', 'Greece', 'Holy See', 'Hong Kong', 'Hong Kong SAR', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iran (Islamic Republic of)', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Japan', 'Jordan', 'Kuwait', 'Latvia', 'Lebanon', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao SAR', 'Macau', 'Mainland China', 'Malaysia', 'Maldives', 'Malta', 'Martinique', 'Mexico', 'Moldova', 'Monaco', 'Mongolia

In [163]:
grouped_df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1029 entries, 0 to 1028
Data columns (total 7 columns):
Active            990 non-null float64
Confirmed         990 non-null float64
Country/Region    1029 non-null object
Date              1029 non-null object
Deaths            990 non-null float64
Log Confirmed     990 non-null float64
Recovered         990 non-null float64
dtypes: float64(5), object(2)
memory usage: 64.3+ KB


In [140]:
grouped_df = grouped_df_2

In [141]:
grouped_df.sort_values(['Country/Region', 'Date'], inplace=True)

In [144]:
grouped_df.loc[grouped_df['Country/Region']=='Azerbaijan']

,Active,Confirmed,Country/Region,Date,Deaths,Log Confirmed,Recovered
57,3.0,3.0,Azerbaijan,2020-03-01 00:00:00,0.0,1.386294,0.0
58,6.0,6.0,Azerbaijan,2020-03-05 00:00:00,0.0,1.945910,0.0
59,9.0,9.0,Azerbaijan,2020-03-07 00:00:00,0.0,2.302585,0.0
60,11.0,11.0,Azerbaijan,2020-03-10 00:00:00,0.0,2.484907,0.0


In [95]:
china_dates = list(grouped_df[grouped_df['Country/Region']=='Mainland China']['Date'])

In [33]:
@interact(Country = df['Country/Region'].sort_values().unique(), Confirmed=True, Active=True, Recovered=True, Deaths=True)
def viz(Country, Confirmed, Active, Recovered, Deaths):
    fig = px.line(title='Cases Over Time')
    if Confirmed:
        country_df = pd.DataFrame(df.loc[df['Country/Region']==Country].groupby('Date')['Confirmed'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Confirmed'], mode='lines', name='Confirmed Cases')
    if Active:
        country_df = pd.DataFrame(df.loc[df['Country/Region']==Country].groupby('Date')['Active'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Active'], mode='lines', name='Active Cases')
    if Recovered:
        country_df = pd.DataFrame(df.loc[df['Country/Region']==Country].groupby('Date')['Recovered'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Recovered'], mode='lines', name='Recovered Cases')
    if Deaths:
        country_df = pd.DataFrame(df.loc[df['Country/Region']==Country].groupby('Date')['Deaths'].agg('sum'))
        fig.add_scatter(x = country_df.index, y = country_df['Deaths'], mode='lines', name='Deceased Cases')
    fig.update_xaxes(title='Date', range=[dt.date(2020, 2, 1), dt.datetime.now()])
    fig.show()

interactive(children=(Dropdown(description='Country', options=(' Azerbaijan', 'Afghanistan', 'Albania', 'Alger…

In [75]:
fig = px.scatter_geo(grouped_df, locations='Country/Region', locationmode='country names', hover_name='Country/Region',
                     size='Log Confirmed', animation_frame='Date')
fig.show()